# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [4]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI. Founded by renowned AI expert Andrew Ng, this platform offers top-tier courses on deep learning, neural networks, and more. Whether you are a beginner or an expert, DeepLearning.AI has something for everyone. With hands-on projects and expert instruction, you can upskill in AI and advance your career. Join the thriving community of learners worldwide and unlock the potential of AI with DeepLearning.AI. Start your journey to mastering artificial intelligence today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [7]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI. Founded by renowned AI expert Andrew Ng, this platform offers top-tier courses on deep learning, neural networks, and more. Whether you are a beginner or an expert, DeepLearning.AI has something for everyone. With hands-on projects and expert instruction, you can upskill in AI and advance your career. Join the thriving community of learners worldwide and unlock the potential of AI with DeepLearning.AI. Start your journey to mastering artificial intelligence today!

--------------------------------------------------------------------------------
Critic (to Writer):

The blogpost provides a compelling overview of the D

## Nested chat

In [9]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [10]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [11]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [12]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [14]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [15]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Dive into the world of artificial intelligence with DeepLearning.AI. Founded by renowned AI expert Andrew Ng, this platform offers top-tier courses on deep learning, neural networks, and more. Whether you are a beginner or an expert, DeepLearning.AI has something for everyone. With hands-on projects and expert instruction, you can upskill in AI and advance your career. Join the thriving community of learners worldwide and unlock the potential of AI with DeepLearning.AI. Start your journey to mastering artificial intelligence today!

--------------------------------------------------------------------------------

************************************************************************


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregated feedback from all reviewers:

1. The content effectively introduces DeepLearning.AI, mentioning key aspects like courses on deep learning and neural networks, the founder Andrew Ng, and the opportunities for learners to upskill and advance their careers.
2. Incorporating relevant keywords strategically within the content can further enhance its SEO performance.
3. Including internal links to related content can help improve overall search visibility.
4. Ensure that claims regarding career advancement and upskilling are backed up by reliable sources or data to avoid misleading information.
5. Ensure transparency about the costs associated with participating in DeepLearning.AI courses to prevent potential financial conflicts of interest.

Final Suggestions:
- Consider integrating more keywords related to AI and online learning for better SEO performance.
- Provide veri

## Get the summary

In [16]:
print(res.summary)

Title: Unveiling the Power of DeepLearning.AI

Embark on a transformative journey into artificial intelligence with DeepLearning.AI, a leading platform founded by AI luminary Andrew Ng. Access cutting-edge courses on deep learning and neural networks designed for learners at all levels. Through practical projects and expert guidance, you can hone your AI skills and propel your career forward. Join a global community of AI enthusiasts and unlock endless possibilities with DeepLearning.AI. Begin your path to AI mastery today! Find out more about DeepLearning.AI's courses and opportunities for career growth while exploring the exciting realm of artificial intelligence.


In [17]:
print(res.cost)

{'usage_including_cached_inference': {'total_cost': 0.0016135000000000004, 'gpt-3.5-turbo-0125': {'cost': 0.0016135000000000004, 'prompt_tokens': 1175, 'completion_tokens': 684, 'total_tokens': 1859}}, 'usage_excluding_cached_inference': {'total_cost': 0.0011905000000000002, 'gpt-3.5-turbo-0125': {'cost': 0.0011905000000000002, 'prompt_tokens': 1001, 'completion_tokens': 460, 'total_tokens': 1461}}}
